# Handcrafted Features 

## Imports

In [ ]:

########################### (https://github.com/curiousily/TensorFlow-on-Android-for-Human-Activity-Recognition-with-LSTMs/blob/master/human_activity_recognition.ipynb) imports
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt


from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

%matplotlib inline
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42

#!pip install imbalanced-learn

from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from keras import backend as K
from sklearn.utils import shuffle

from matplotlib.pyplot import figure


########################### (https://github.com/curiousily/TensorFlow-on-Android-for-Human-Activity-Recognition-with-LSTMs/blob/master/human_activity_recognition.ipynb) imports
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt


from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


# This is for RandomSearchCV error/delay  speeds up the searches 
from sklearn.externals.joblib import parallel_backend 



# Upload Data 

---------------------------------



Train

In [ ]:
from google.colab import files
uploaded_signal_train = files.upload()

Test

In [ ]:
from google.colab import files
uploaded_signal_test = files.upload()

### Reading in segments & labels

Train

In [ ]:
# getting keys, which is file names of npy 
list_of_dataframes_train = [key for key in uploaded_signal_train.keys()]


# set up list to hold all loaded npy 
all_dataframe_train = [] 


for i in range(len(list_of_dataframes_train)):

    # load in the data 
    dataframe_train = pd.read_csv(list_of_dataframes_train[i])


    # append the data to 'all' list
    all_dataframe_train.append(dataframe_train) 


all_df_train = pd.concat(all_dataframe_train)

Test

In [ ]:
# getting keys, which is file names of npy 
list_of_dataframes_test = [key for key in uploaded_signal_test.keys()]


# set up list to hold all loaded npy 
all_dataframe_test = [] 


for i in range(len(list_of_dataframes_test)):

    # load in the data 
    dataframe_test = pd.read_csv(list_of_dataframes_test[i])


    # append the data to 'all' list
    all_dataframe_test.append(dataframe_test) 


all_df_test = pd.concat(all_dataframe_test)


# Quick Look 

In [ ]:
all_df_train.head()

In [ ]:
all_df_test.head()

# Train Test Split 

In [ ]:
# Getting X_train & y_train'
X_train = all_df_train.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run', 'X_Acc_Move_FFT_EnergyBands', 	'Y_Acc_Move_FFT_EnergyBands', 	'Z_Acc_Move_FFT_EnergyBands'	], axis = 1)
y_train = all_df_train['Label_segment'].values

In [ ]:
# Getting X_train & y_train
X_test = all_df_test.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run',  'X_Acc_Move_FFT_EnergyBands', 	'Y_Acc_Move_FFT_EnergyBands', 	'Z_Acc_Move_FFT_EnergyBands'], axis = 1)
y_test = all_df_test['Label_segment'].values

--------------------------------------

Standard Scale 

In [ ]:
# Set up ss for non-shuffled data
ss = StandardScaler()


# fit train & transform test
X_train_scale = ss.fit_transform(X_train)
X_test_scale = ss.transform(X_test)


<br>

# Tuning 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score, log_loss , recall_score , f1_score, precision_score , roc_auc_score, balanced_accuracy_score
from sklearn.model_selection import GridSearchCV , RandomizedSearchCV

---------------------------

MLP Classifier Tuning 

- alpha 
- hidden layers
- activation
- solver
- learning rate 

**Alpha**

In [ ]:
# Alpha values 

alphas = [0, 0.0001 , 0.001 , 0.01 , 0.1 ]

ein=[]
eout=[]

# small little tweak to return Eout min error, setting min_error extremely high so anything is better than np.inf
min_error = 0
i = 0 

for a in alphas:

  print(i)

  # initialize model
  clf = MLPClassifier(alpha=a)

  # fit model
  clf.fit(X_train_scale, y_train)

  # get in sample/train f1-score 
  train_preds = clf.predict(X_train_scale)
  train_f1 = f1_score(train_preds, y_train, average='weighted')
  ein.append(train_f1) 
             
  # get out of sample error 
  test_preds = clf.predict(X_test_scale)
  test_f1 = f1_score(test_preds, y_test, average='weighted')
  eout.append(test_f1) 

  i += 1

  # if current error is less than minimum, update it to be minimum error and store the value of lambda for it     
  if test_f1 > min_error:
      min_error = test_f1
      alpha_min = a


In [ ]:
print(f"Max F1 score {min_error} : alpha value {alpha_min} " )

In [ ]:
plt.plot(alphas , eout ,  label = 'F1 Score'  )

plt.xlabel("Alpha")
plt.ylabel("Test F1 Score")
plt.title("Test F1 Score Vs Alpha")

plt.legend()
plt.show()

**Hidden Layer**

In [ ]:
# Alpha values 

hidden_layers = [ (50, 20) , (50,10,50), (100, 50 , 20) , (200, 50 , 100) ]

ein=[]
eout=[]

# small little tweak to return Eout min error, setting min_error extremely high so anything is better than np.inf
min_error = 0
i = 0 

for layer in hidden_layers:

  print(i)

  # initialize model
  clf = MLPClassifier(hidden_layer_sizes= layer)

  # fit model
  clf.fit(X_train_scale, y_train)

  # get in sample/train f1-score 
  train_preds = clf.predict(X_train_scale)
  train_f1 = f1_score(train_preds, y_train, average='weighted')
  ein.append(train_f1) 
             
  # get out of sample error 
  test_preds = clf.predict(X_test_scale)
  test_f1 = f1_score(test_preds, y_test, average='weighted')
  eout.append(test_f1) 

  i += 1

  # if current error is less than minimum, update it to be minimum error and store the value of lambda for it     
  if test_f1 > min_error:
      min_error = test_f1
      best_layer = layer

In [ ]:
for i in range(len(eout)):

  print( ein[i] , eout[i] , hidden_layers[i])

**Now Search**

In [ ]:
# Search Space

parameter_space = {
    'hidden_layer_sizes': [(50, 20) , (200, 50, 100) , (100, 50 ,20)],
    'activation': ['tanh', 'relu', 'logistic'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': np.arange(0.00001 , 0.0005, 0.001, 0.01, 0.1),
    'learning_rate': ['constant','adaptive']
    
    }





with parallel_backend('threading'):

  gsearch_mlp = RandomizedSearchCV(estimator = MLPClassifier(),
                        param_distributions = parameter_space,  
                        scoring = 'f1_weighted',                
                        cv = 3,
                        n_jobs = 4,
                        verbose = 5)
  
  grid_search_mlp = gsearch_mlp.fit(X_train_scale, y_train)
  
print("MLP Finished")

In [ ]:
gsearch_mlp.best_score_

In [ ]:
gsearch_mlp.best_params_

In [ ]:
gs_mlp_model = gsearch_mlp.best_estimator_

In [ ]:
preds = gs_mlp_model.predict(X_test_scale)
train_preds = gs_mlp_model.predict(X_train_scale)

print("-------------------Train------------------")
print("Train Balanced Accuracy : \t" , balanced_accuracy_score(y_train, train_preds))
print("Train F1 Score : \t\t" , f1_score(y_train, train_preds , average='weighted'))
print("Train Precision Score : \t" , precision_score(y_train, train_preds , average='weighted'))
print("Train Recall Score : \t\t" , recall_score(y_train, train_preds , average='weighted'))

print("\n\n-------------------Test------------------")
print("Test Balanced Accuracy : \t" , balanced_accuracy_score(y_test, preds))
print("Test F1 Score : \t\t" , f1_score(y_test, preds , average='weighted'))
print("Test Precision Score : \t\t" , precision_score(y_test, preds , average='weighted'))
print("Test Recall Score : \t\t" , recall_score(y_test, preds , average='weighted'))

----------------------------------

Decision Tree Tuning (https://medium.com/@mohtedibf/indepth-parameter-tuning-for-decision-tree-6753118a03c3)

- Max Depth
- Min Sample Split
- Min Samples Leaf 
- Criterion
- Max Feature


**Depth**

In [ ]:

# Alpha values 

max_depth = [ 1, 5,10, 12, 15, 18, 20 , 30 ]

ein=[]
eout=[]

# small little tweak to return Eout min error, setting min_error extremely high so anything is better than np.inf
min_error = 0
i = 0 

for depth in max_depth:

  print(i)

  # initialize model
  clf = DecisionTreeClassifier(max_depth= depth)

  # fit model
  clf.fit(X_train_scale, y_train)

  # get in sample/train f1-score 
  train_preds = clf.predict(X_train_scale)
  train_f1 = f1_score(train_preds, y_train, average='weighted')
  ein.append(train_f1) 
             
  # get out of sample error 
  test_preds = clf.predict(X_test_scale)
  test_f1 = f1_score(test_preds, y_test, average='weighted')
  eout.append(test_f1) 

  i += 1

  # if current error is less than minimum, update it to be minimum error and store the value of lambda for it     
  if test_f1 > min_error:
      min_error = test_f1
      best_depth = depth

In [ ]:
print(f"Max F1 score {min_error} : depth value {best_depth} " )

In [ ]:
for i in range(len(eout)):

  print("Training F1-Score", ein[i] , "\tTesting F1-Score" , eout[i] , "\tMax Depth", max_depth[i])

**Min Sample Split**

In [ ]:
# Alpha values 

sample_split = [ 2 , 10 , 25 , 50 , 100 , 150 , 200 ]

ein=[]
eout=[]

# small little tweak to return Eout min error, setting min_error extremely high so anything is better than np.inf
min_error = 0
i = 0 

for sam_split in sample_split:

  print(i)

  # initialize model
  clf = DecisionTreeClassifier( min_samples_split= sam_split)

  # fit model
  clf.fit(X_train_scale, y_train)

  # get in sample/train f1-score 
  train_preds = clf.predict(X_train_scale)
  train_f1 = f1_score(train_preds, y_train, average='weighted')
  ein.append(train_f1) 
             
  # get out of sample error 
  test_preds = clf.predict(X_test_scale)
  test_f1 = f1_score(test_preds, y_test, average='weighted')
  eout.append(test_f1) 

  i += 1

  # if current error is less than minimum, update it to be minimum error and store the value of lambda for it     
  if test_f1 > min_error:
      min_error = test_f1
      best_sample_split = sam_split

In [ ]:
print(f"Max F1 score {min_error} : Min Sample Split best value {best_sample_split} " )

In [ ]:
for i in range(len(eout)):

  print("Training F1-Score", ein[i] , "\tTesting F1-Score" , eout[i] , "\tMin Sample Split", sample_split[i])

**Min Sample Leaf**

In [ ]:

# Alpha values 

sample_leaf = [1,2,3,4,5,6,7,8,9,10,25,50]

ein=[]
eout=[]

# small little tweak to return Eout min error, setting min_error extremely high so anything is better than np.inf
min_error = 0
i = 0 

for sam_leaf in sample_leaf:

  print(i)

  # initialize model
  clf = DecisionTreeClassifier(  min_samples_leaf=  sam_leaf)

  # fit model
  clf.fit(X_train_scale, y_train)

  # get in sample/train f1-score 
  train_preds = clf.predict(X_train_scale)
  train_f1 = f1_score(train_preds, y_train, average='weighted')
  ein.append(train_f1) 
             
  # get out of sample error 
  test_preds = clf.predict(X_test_scale)
  test_f1 = f1_score(test_preds, y_test, average='weighted')
  eout.append(test_f1) 

  i += 1

  # if current error is less than minimum, update it to be minimum error and store the value of lambda for it     
  if test_f1 > min_error:
      min_error = test_f1
      best_sample_leaf =  sam_leaf

In [ ]:
print(f"Max F1 score {min_error} : min sample leaf best value {best_sample_leaf} " )

In [ ]:
for i in range(len(eout)):

  print("Training F1-Score", ein[i] , "\tTesting F1-Score" , eout[i] , "\tmin sample leaf", sample_leaf[i])

<br>

**Tuning**

In [ ]:
# Hyper parameters range intialization for tuning 

parameters={

            "criterion":["gini","entropy"],
            "max_depth" : [None, 5,10, 12, 15, 18, 20 ],
           "min_samples_leaf":[1 , 3,5,6,7,8,9,10],
            "min_samples_split": [2 ,3, 5, 10 , 25 , 50 , 100 , 150 , 200 ],
          #  "max_features":["auto","log2","sqrt",None],
          #  "max_leaf_nodes":[None,10,20,30,40,50,60,70,80,90]
            }

In [ ]:
# Tuning 


gsearch_dtc = RandomizedSearchCV(estimator = DecisionTreeClassifier(),
                      param_distributions = parameters,  
                      scoring = 'f1_weighted',                
                      cv = 3,
                      n_jobs = 4,
                      verbose = 4)

grid_search_dtc = gsearch_dtc.fit(X_train_scale, y_train)
  
print("DT Classifier Finished")

In [ ]:
gsearch_dtc.best_score_

In [ ]:
gsearch_dtc.best_params_

In [ ]:
gs_dtc_model = gsearch_dtc.best_estimator_

In [ ]:
preds = gs_dtc_model.predict(X_test_scale)
train_preds = gs_dtc_model.predict(X_train_scale)

print("-------------------Train------------------")
print("Train Balanced Accuracy : \t" , balanced_accuracy_score(y_train, train_preds))
print("Train F1 Score : \t\t" , f1_score(y_train, train_preds , average='weighted'))
print("Train Precision Score : \t" , precision_score(y_train, train_preds , average='weighted'))
print("Train Recall Score : \t\t" , recall_score(y_train, train_preds , average='weighted'))

print("\n\n-------------------Test------------------")
print("Test Balanced Accuracy : \t" , balanced_accuracy_score(y_test, preds))
print("Test F1 Score : \t\t" , f1_score(y_test, preds , average='weighted'))
print("Test Precision Score : \t\t" , precision_score(y_test, preds , average='weighted'))
print("Test Recall Score : \t\t" , recall_score(y_test, preds , average='weighted'))

---------------------------------------

Logisitic Regression 

- penalty 
- C value 
- class_weight
- solver
- multi_class


<br>

**C Value**

In [ ]:

# Alpha values 

C_vals = [100, 10, 1.0, 0.1, 0.01]

ein=[]
eout=[]

# small little tweak to return Eout min error, setting min_error extremely high so anything is better than np.inf
min_error = 0
i = 0 

for c in C_vals:

  print(i)

  # initialize model
  clf = LogisticRegression( C = c)

  # fit model
  clf.fit(X_train_scale, y_train)

  # get in sample/train f1-score 
  train_preds = clf.predict(X_train_scale)
  train_f1 = f1_score(train_preds, y_train, average='weighted')
  ein.append(train_f1) 
             
  # get out of sample error 
  test_preds = clf.predict(X_test_scale)
  test_f1 = f1_score(test_preds, y_test, average='weighted')
  eout.append(test_f1) 

  i += 1

  # if current error is less than minimum, update it to be minimum error and store the value of lambda for it     
  if test_f1 > min_error:
      min_error = test_f1
      best_c =  c

In [ ]:
print(f"Max F1 score {min_error} : c value {best_c} " )

In [ ]:
for i in range(len(eout)):

  print("Training F1-Score", ein[i] , "\tTesting F1-Score" , eout[i] , "\tC values", C_vals[i])

**Max Iters**

In [ ]:
# Alpha values 

max_iters = [100, 200, 500]

ein=[]
eout=[]

# small little tweak to return Eout min error, setting min_error extremely high so anything is better than np.inf
min_error = 0
i = 0 

for val in max_iters:

  print(i)

  # initialize model
  clf = LogisticRegression( max_iter=  val)

  # fit model
  clf.fit(X_train_scale, y_train)

  # get in sample/train f1-score 
  train_preds = clf.predict(X_train_scale)
  train_f1 = f1_score(train_preds, y_train, average='weighted')
  ein.append(train_f1) 
             
  # get out of sample error 
  test_preds = clf.predict(X_test_scale)
  test_f1 = f1_score(test_preds, y_test, average='weighted')
  eout.append(test_f1) 

  i += 1

  # if current error is less than minimum, update it to be minimum error and store the value of lambda for it     
  if test_f1 > min_error:
      min_error = test_f1
      best_val =  val

In [ ]:
print(f"Max F1 score {min_error} : c value {best_val} " )

In [ ]:
for i in range(len(eout)):

  print("Training F1-Score", ein[i] , "\tTesting F1-Score" , eout[i] , "\tC values", max_iters[i])

In [ ]:
# Hyper parameters range intialization for tuning 

parameters={

            "penalty":["l1","l2"  ],
            "C" : [100, 10, 1, 0.1, 0.01],
            "class_weight":[None, 'balanced'],
            "solver": ['newton-cg', 'lbfgs', 'liblinear'],
            "multi_class":['auto', 'ovr', 'multinomial']

            }


# Tuning 

gsearch_lr = RandomizedSearchCV(estimator = LogisticRegression(),
                      param_distributions = parameters,  
                      scoring = 'f1_weighted',                
                      cv = 2,
                      n_jobs = 4,
                      verbose = 5)

grid_search_lr = gsearch_lr.fit(X_train_scale, y_train)
  
print("LR Classifier Finished")

In [ ]:
gsearch_lr.best_score_

In [ ]:
gsearch_lr.best_params_

In [ ]:
gs_lr_model = gsearch_lr.best_estimator_

In [ ]:
preds = gs_lr_model.predict(X_test_scale)
train_preds = gs_lr_model.predict(X_train_scale)

print("-------------------Train------------------")
print("Train Balanced Accuracy : \t" , balanced_accuracy_score(y_train, train_preds))
print("Train F1 Score : \t\t" , f1_score(y_train, train_preds , average='weighted'))
print("Train Precision Score : \t" , precision_score(y_train, train_preds , average='weighted'))
print("Train Recall Score : \t\t" , recall_score(y_train, train_preds , average='weighted'))

print("\n\n-------------------Test------------------")
print("Test Balanced Accuracy : \t" , balanced_accuracy_score(y_test, preds))
print("Test F1 Score : \t\t" , f1_score(y_test, preds , average='weighted'))
print("Test Precision Score : \t\t" , precision_score(y_test, preds , average='weighted'))
print("Test Recall Score : \t\t" , recall_score(y_test, preds , average='weighted'))

-----------------------------

Neural Nets 

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

In [ ]:
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from matplotlib import pyplot

from keras.optimizers import SGD , Adam , Adamax , Adadelta , Adagrad

from keras.optimizers import Adam
from keras import backend as K
import time
import tensorflow as tf

from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
# Getting X_train & y_train'
X_train = all_df_train.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run', 'X_Acc_Move_FFT_EnergyBands', 	'Y_Acc_Move_FFT_EnergyBands', 	'Z_Acc_Move_FFT_EnergyBands'	], axis = 1)
#X_train = all_df_train.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run'	], axis = 1)

y_train = all_df_train['Label_segment'].values


# convert to dummy category
y_train_dum = pd.get_dummies(y_train)

print(f'\nDummy labels Shape {y_train_dum.shape}')

In [ ]:
# Getting X_train & y_train

X_test = all_df_test.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run',  'X_Acc_Move_FFT_EnergyBands', 	'Y_Acc_Move_FFT_EnergyBands', 	'Z_Acc_Move_FFT_EnergyBands'], axis = 1)
#X_test = all_df_test.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run'	], axis = 1)

y_test = all_df_test['Label_segment'].values

# convert to dummy category
y_test_dum = pd.get_dummies(y_test)

print(f'\nDummy labels Shape {y_test_dum.shape}')

In [ ]:
# Set up ss for non-shuffled data
ss = StandardScaler()

# fit for non-shuffled
X_train_scale = ss.fit_transform(X_train)
X_test_scale = ss.transform(X_test)

In [ ]:
n_outputs =  y_train_dum.shape[1]

In [ ]:
n_outputs

In [ ]:

n_outputs =  y_train_dum.shape[1]

# set up model
model = Sequential()

#input layer
model.add(Dense(100, activation='relu' , input_shape = (X_train_scale.shape[1], )))

# hidden layers
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))


# output layer
model.add(Dense(6, activation='softmax'))

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
# Function to create model, required for KerasClassifier
def create_model(optimizer = Adam , neurons=1, dropout = 0 , learn_rate = 0.001 , activation = 'relu' , ):

  # create model

  # set up model
  model = Sequential()

  #input layer
  model.add(Dense(neurons, activation= activation , input_shape = (X_train_scale.shape[1], )))

  # hidden layers
  model.add(Dense(neurons, activation= activation))
  model.add(Dropout(dropout))
  model.add(Dense(neurons, activation= activation))


  # output layer
  model.add(Dense(6, activation='softmax'))

  opt = optimizer(learning_rate=learn_rate)

  # compile model
  model.compile(loss='categorical_crossentropy', optimizer = opt, metrics=[tf.keras.metrics.CategoricalAccuracy()])


  return model

In [ ]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=200, verbose=0)

In [ ]:
# define the grid search parameters

neurons = [ 10, 50 , 100]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4]
learn_rate = [0.001 , 0.01]
optimisers = [SGD, Adam, Adamax]
batch_sizes = [50,200,1000]
activations = ['relu' , 'tanh']
epochs = [10, 100 , 250]


param_grid = dict(dropout=dropout_rate, neurons =neurons , learn_rate = learn_rate , activation = activations , optimizer =optimisers , batch_size = batch_sizes , epochs = epochs) 

grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=-1, cv=2 ,  scoring = 'f1_weighted' , verbose = 5 )

grid_result = grid.fit(X_train_scale, y_train)


# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
grid_result.best_score_

In [ ]:
grid_result.best_params_

In [ ]:
gs_NN_model = grid_result.best_estimator_

In [ ]:
preds = gs_NN_model.predict(X_test_scale)
train_preds = gs_NN_model.predict(X_train_scale)

print("-------------------Train------------------")
print("Train Balanced Accuracy : \t" , balanced_accuracy_score(y_train, train_preds))
print("Train F1 Score : \t\t" , f1_score(y_train, train_preds , average='weighted'))
print("Train Precision Score : \t" , precision_score(y_train, train_preds , average='weighted'))
print("Train Recall Score : \t\t" , recall_score(y_train, train_preds , average='weighted'))

print("\n\n-------------------Test------------------")
print("Test Balanced Accuracy : \t" , balanced_accuracy_score(y_test, preds))
print("Test F1 Score : \t\t" , f1_score(y_test, preds , average='weighted'))
print("Test Precision Score : \t\t" , precision_score(y_test, preds , average='weighted'))
print("Test Recall Score : \t\t" , recall_score(y_test, preds , average='weighted'))

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# set up labels 
LABELS = ['Go', 'Turn1',  'Turn2' , 'Walk1', 'Walk2', 'Sit']


In [ ]:
# Test 
print(classification_report(y_test, preds , target_names=LABELS))

In [ ]:
n_ouputs = y_train_dum.shape[1]

# set up model
model = Sequential()

# input layer
model.add(Dense(100, activation='relu' , input_shape = (X_train_scale.shape[1], )))

# hidden layers
model.add(Dense(100, activation='relu' ))
model.add(Dropout(0.3))
model.add(Dense(100, activation='relu' ))

# output layer 
model.add(Dense(n_outputs , activation='softmax'))

opt = Adam(learning_rate=0.001)

# compile model 
model.compile(loss= 'categorical_crossentropy' , optimizer= opt , metrics= [tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
confusion_matrix = metrics.confusion_matrix(y_test, preds)
confusion_matrix_t = metrics.confusion_matrix(y_train, train_preds )

In [ ]:
# Test 
plt.figure(figsize=(14, 10))
sns.heatmap(confusion_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True , fmt="d");
plt.title("Test Data Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();

In [ ]:
# Train
print(classification_report(y_train, train_preds , target_names=LABELS))

In [ ]:
# Train
plt.figure(figsize=(14, 10))
sns.heatmap(confusion_matrix_t, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
plt.title("Train Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();